In [3]:
import numpy as np
import pandas as pd

from collections import OrderedDict

In [168]:
class AHP():
    RI = {1: 0, 2: 0, 3: 0.58, 4: 0.9, 5: 1.12, 6: 1.24, 7: 1.32, 8: 1.41, 9: 1.45}
    consistency = False
    priority_vec = None
    compete = False
    normal = False
    sublayer = None
    
    def __init__(self, name, size):
        self.name = name
        self.size = size
        self.matrix = np.zeros([size,size])
        self.criteria = [None] * size
               
    def update_matrix(self, mat, automated=True):
        if not ((mat.shape[0] == mat.shape[1]) and (mat.ndim == 2)):
            raise Exception('Input matrix must be squared.')
        if self.size != len(self.criteria):
            self.criteria = [None] * size
        self.matrix = mat
        self.size = mat.shape[0]
        self.consistency = False
        self.normal = False
        self.priority_vec = None
        if automated:
            self.rank()
        
    def input_prioriry_vec(self, vec):
        if not (vec.shape[1] == 1) and (vec.shape[0] == self.size) and (vec.ndim == 2):
            raise Exception('The size of input priority vector is not compatable.')
        self.priority_vec = vec
        self.output = self.priority_vec / self.priority_vec.sum()
        self.consistency = True
        self.normal = True
        
    def rename(self, name):
        self.name = name
        
    def update_criteria(self, criteria):
        if len(criteria) == self.size:
            self.criteria = criteria
        else:
            raise Exception('Input doesn\'t match the number of criteria.')
            
    def add_layer(self, alternative):
        if not self.criteria:
            raise Exception('Please input criterias before adding new layer.')
        self.compete  = False
        self.sublayer = OrderedDict()
        self.alternative = alternative
        for i in range(self.size):
            self.sublayer[self.criteria[i]] = AHP(self.criteria[i], len(alternative))
            self.sublayer[self.criteria[i]].update_criteria(self.alternative)
            
    def normalize(self):
        if self.normal:
            pass
        col_sum = self.matrix.sum(axis = 0)
        try:
            self.matrix = self.matrix / col_sum
        except:
            raise Exception('Error when normalize on columns.')
        else:
            self.nomral = True
            self.priority_vec = self.matrix.sum(axis = 1).reshape(-1,1)
    
    def rank(self):
        if self.consistency:
            df = pd.DataFrame(data = self.output, index = self.criteria, columns=[self.name])
            # display(df)
            return df
        if not self.normal:
            self.normalize()
        Ax = self.matrix.dot(self.priority_vec)
        eigen_val = (Ax / self.priority_vec).mean()
        CI = (eigen_val - self.size) / (self.size - 1)
        CR = CI / self.RI[self.size]
        if CR < 0.1:
            self.consistency = True
            self.output = self.priority_vec / self.priority_vec.sum()
            self.df_out = pd.DataFrame(data = self.output, index = self.criteria, columns=[self.name])
            return self.df_out
        else:
            raise Exception('The consistency for desicion is not sufficient.')
            
    def make_decision(self):
        if not self.consistency:
            self.rank()
        if not self.compete:
            temp = True
            arrays = []
            interresults = []
            colnames = []
            for item in self.sublayer.values():
                tempdf = item.rank()
                temp = temp and item.consistency
                if temp:
                    arrays.append(item.output)
                    interresults.append(tempdf.values)
                    colnames.append(list(tempdf.columns))
                else:
                    raise Exception('Please check the AHP for {}'.format(item.name))
            if temp:
                self.compete = True
            else:
                pass
            self.recommendation = np.concatenate(arrays, axis = 1).dot(self.output)
            print ("1---->",self.recommendation)
            self.inter = np.concatenate(interresults,axis=1)
            print ("2---->",self.inter)
            self.interfinal = np.concatenate((self.inter,self.recommendation),axis=1)
            print ("3---->",self.interfinal)
            self.flat_list = [item for sublist in colnames for item in sublist]
            self.flat_list.append('AHP Score')
                       
        self.df_decision = pd.DataFrame(data = self.interfinal, index = self.alternative, columns = self.flat_list)
        self.df_decision.index.name = 'Alternative'
        self.df_decision['rank'] = self.df_decision['AHP Score'].rank(ascending = False)
        return self.df_decision

In [169]:
car = AHP('car', 3)
car.update_matrix(np.array([
    [1,0.5,3],
    [2,1,4],
    [0.3333,0.25,1]
]))

car.update_criteria(['Style', 'Cost', 'Fuel Economy'])
car.rank()

,car
Style,0.320239
Cost,0.557145
Fuel Economy,0.122616


In [170]:
m1 = [
    [1,0.25,4,0.1667],
    [4,1,4,0.25],
    [0.25,0.25,1,0.2],
    [6,4,5,1]
]

m2 = [
    [1,2,5,1],
    [0.5,1,3,2],
    [0.2,0.3333,1,0.25],
    [1,0.5,4,1]
]

m3 = [
    [34],
    [27],
    [24],
    [28]   
]

In [171]:
np.array(m3).shape[0], np.array(m3).shape[1], np.array(m3).ndim 

(4, 1, 2)

In [172]:
car.add_layer(['Civic', 'i20', 'Escort', 'Alto'])

car.sublayer['Style'].update_matrix(np.array(m1))

car.sublayer['Cost'].update_matrix(np.array(m2))

car.sublayer['Fuel Economy'].update_matrix(np.array(m2))

car.make_decision()

1----> [[0.29883841]
 [0.27326824]
 [0.07147388]
 [0.35641947]]
2----> [[0.13079225 0.37800588 0.37800588]
 [0.244431   0.28685362 0.28685362]
 [0.06570353 0.07419232 0.07419232]
 [0.55907322 0.26094818 0.26094818]]
3----> [[0.13079225 0.37800588 0.37800588 0.29883841]
 [0.244431   0.28685362 0.28685362 0.27326824]
 [0.06570353 0.07419232 0.07419232 0.07147388]
 [0.55907322 0.26094818 0.26094818 0.35641947]]


,Style,Cost,Fuel Economy,AHP Score,rank
Alternative,,,,,
Civic,0.130792,0.378006,0.378006,0.298838,2.0
i20,0.244431,0.286854,0.286854,0.273268,3.0
Escort,0.065704,0.074192,0.074192,0.071474,4.0
Alto,0.559073,0.260948,0.260948,0.356419,1.0


In [52]:
a

,AHP Score,rank
Alternative,,
Civic,0.289382,2.0
i20,0.267393,3.0
Escort,0.088419,4.0
Alto,0.354806,1.0


In [59]:
b[0]

,Style
Civic,0.130792
i20,0.244431
Escort,0.065704
Alto,0.559073


In [60]:
b[0].values

array([[0.13079225],
       [0.244431  ],
       [0.06570353],
       [0.55907322]])

In [64]:
array = []
array.append(b[0])
array.append(b[1])
array.append(b[2])
array

[           Style
 Civic   0.130792
 i20     0.244431
 Escort  0.065704
 Alto    0.559073,
             Cost
 Civic   0.378006
 i20     0.286854
 Escort  0.074192
 Alto    0.260948,
         Fuel Economy
 Civic       0.300885
 i20         0.238938
 Escort      0.212389
 Alto        0.247788]

In [66]:
recommendation = np.concatenate(array, axis = 1)
pd.DataFrame(data = recommendation, columns = ['style', 'cost', 'fuel economy'])

,style,cost,fuel economy
0,0.130792,0.378006,0.300885
1,0.244431,0.286854,0.238938
2,0.065704,0.074192,0.212389
3,0.559073,0.260948,0.247788


In [134]:
a = ['q','w']

In [135]:
a

['q', 'w']

In [136]:
a.append('r')

In [137]:
a

['q', 'w', 'r']